<a href="https://colab.research.google.com/github/Y0shicon/SOP/blob/main/SOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDVI Calculation using the sentinel data

In [ ]:
import ee
import geemap

In [ ]:
bits_coords = [28.358899, 75.587724]
point=ee.Geometry.Point(bits_coords[1], bits_coords[0]);

In [ ]:
Map = geemap.Map(center=bits_coords, zoom=15)
Map

Map(center=[28.358899, 75.587724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [ ]:
#location point
data = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point)

# Retrieve an image with the desired bands
image = data.filterDate("2019-01-01", "2019-12-31").first()

ndvi=image.expression(
"(NIR - RED) / (NIR + RED)",
{"NIR":image.select("B8"),
"RED":image.select("B4")});

display={
    "min":0,
    "max":1,
    "palette":'terrain'
}

Map.addLayer(ndvi, display, "NDVI");
Map.centerObject(point, 13)
Map

Map(bottom=55065.0, center=[28.358899, 75.587724], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
m = geemap.Map(center=bits_coords, zoom=5)
image = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 6000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],  # 'terrain'
}
m.add_layer(image, vis_params, "SRTM")
m


Map(center=[28.358899, 75.587724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

# Sentinel Data

In [ ]:
m = geemap.Map()
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterDate("2021-01-01", "2022-01-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)
image = collection.median()

vis = {
    "min": 0.0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],
}

ndvi=image.expression(
"(NIR - RED) / (NIR + RED)",
{"NIR":image.select("B8"),
"RED":image.select("B4")});

display={
    "min":0,
    "max":1,
    "palette":[ 'red','orange', 'yellow','yellowgreen', 'green','black']
}


m.set_center(75.587724, 28.358899, 15)
m.add_layer(image, vis, "Sentinel-2")
m.addLayer(ndvi, display, "NDVI");
m

Map(center=[28.358899, 75.587724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

# Planet SkySat Public Ortho Imagery, Multispectral

In [ ]:
m = geemap.Map()
collection = (
    ee.ImageCollection("SKYSAT/GEN-A/PUBLIC/ORTHO/RGB")
    .filterDate("2021-01-01", "2022-01-01")
)

image = collection.median()

dataset = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')
rgb = dataset.select(['R', 'G', 'B'])

rgbVis = {
  'min': 11.0,
  'max': 190.0,
}

m.add_layer(rgb, rgbVis, "Visible")
m.set_center( -70.892, 41.6555, 15)
# m.set_center( bits_coords[1], bits_coords[0], 15)
m

Map(center=[41.6555, -70.892], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

# Road Segmentation

In [ ]:
## Imports
import os
import sys
import random

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

## Seeding
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

### Path

In [ ]:
dataset_path = "drive/MyDrive/SOP/"
train_path = os.path.join(dataset_path, "train/")

### Data Generator

In [ ]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, "images", id_name) + ".png"
        mask_path = os.path.join(self.path, "masks", id_name) + ".png"

        ## Reading Image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.image_size, self.image_size))

        ##Reading Mask
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (self.image_size, self.image_size))
        mask = np.expand_dims(mask, axis=-1)

        ## Normalizaing
        image = image/255.0
        mask = mask/255.0

        return image, mask

    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size

        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]

        image = []
        mask  = []

        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)

        image = np.array(image)
        mask  = np.array(mask)

        return image, mask

    def on_epoch_end(self):
        pass

    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))


### Hyperparameters

In [ ]:
train_csv = pd.read_csv(dataset_path + "train.csv")
train_ids = train_csv["id"].values

image_size = 128
batch_size = 16

val_data_size = 200

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

In [ ]:
gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
